In [1]:
! pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.5 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.6-py3-none-any.whl size=143237 sha256=76c3d84e69a16d6e6336659adce835d5

In [14]:
import pandas as pd
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,precision_recall_curve,auc
import mlflow
from urllib.parse import urlparse
from markupsafe import escape
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


In [34]:
messages = pd.read_csv('data/SMSSpamCollection',sep='\t',header=None)

In [35]:
col_name = ["label", "message"]
messages.columns= col_name
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 13.4 MB/s eta 0:00:00


In [8]:
import re
import contractions
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string 

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [22]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))


In [33]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:

! pip install wordcloud
from wordcloud import WordCloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
def preprocessing(sentence):

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))

    punct = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
    tokenize = RegexpTokenizer(r'\w+')
  
    sentence = re.sub(punct, "", sentence)
    sentence = contractions.fix(sentence)
    sentence = re.sub(r'https?', "", sentence) 
    sentence = sentence.lower()
    sentence = tokenize.tokenize(sentence)

    sentence = [word for word in sentence if word not in stop_words]
    sentence = [str(lemmatizer.lemmatize(word)) for word in sentence]

    return(sentence)

In [36]:
labels = [0  if x== "ham" else 1 for x in messages.label]

In [37]:
X = [preprocessing(sentence) for sentence in messages.message]

In [38]:
data = pd.DataFrame({'message': X,'labels': labels})

In [39]:
def testTrainValSplit(file, seed):

  from sklearn.model_selection import train_test_split

  X_train, X_test, y_train, y_test = train_test_split(file.message, file.labels, test_size = 0.2, random_state = seed)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = seed)

  train=pd.DataFrame({'X_train': X_train,'y_train': y_train})
  val=pd.DataFrame({'X_val': X_val,'y_val': y_val})
  test=pd.DataFrame({'X_test': X_test,'y_test': y_test})

  train.to_csv('data/train.csv',index=False)
  val.to_csv('data/validation.csv',index=False)
  test.to_csv('data/test.csv',index=False)

In [40]:
testTrainValSplit(data, 0)

In [41]:
test= pd.read_csv('data/test.csv')
train= pd.read_csv('data/train.csv')
valid= pd.read_csv('data/validation.csv')

In [43]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import csv
#from textblob import TextBlob
import pandas
import sklearn
#import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve

In [44]:
count_vectorize = CountVectorizer(analyzer=preprocessing) 
vectorized = count_vectorize.fit_transform(train.X_train)

In [45]:
message = train["X_train"][4]
message4 = count_vectorize.transform([message])

print(message4)
print(message4.shape)

  (0, 1155)	1
  (0, 4211)	1
  (0, 6959)	1
(1, 7303)


In [46]:
message = test["X_test"][4]
message4 = count_vectorize.transform([message])

print(message4)
print(message4.shape)

  (0, 1808)	1
  (0, 1910)	1
  (0, 2652)	1
  (0, 2995)	1
  (0, 3959)	1
(1, 7303)


In [47]:
bag_of_words=count_vectorize.vocabulary_

In [48]:
len(bag_of_words)

7303

In [70]:
X_train= count_vectorize.transform(train.X_train)
X_val = count_vectorize.transform(valid.X_val)
X_test= count_vectorize.transform(test.X_test)

In [71]:
X_train

<4011x7303 sparse matrix of type '<class 'numpy.int64'>'
	with 33760 stored elements in Compressed Sparse Row format>

In [72]:
tfidf_transformer = TfidfTransformer().fit(X_train)
X_train_tfidf = tfidf_transformer.transform(X_train)

In [54]:
X_val_tfidf=tfidf_transformer.transform(X_val)
X_test_tfidf=tfidf_transformer.transform(X_test)

In [55]:
X_val_tfidf

<446x7303 sparse matrix of type '<class 'numpy.float64'>'
	with 3253 stored elements in Compressed Sparse Row format>

##  Model 1 : logistic Regression model

In [56]:
def eval_metrics(actual, pred):
    precision, recall, thresholds = precision_recall_curve(actual, pred)
    auc_precision_recall = auc(recall, precision)
    return (auc_precision_recall)

In [59]:
train

,X_train,y_train
0,"['hi', 'darlin', 'finish', '3', '1', '2', 'pic...",0
1,"['freemsg', 'hi', 'baby', 'wow', 'got', 'new',...",1
2,"['talk', 'ever', 'ok', 'word']",0
3,"['take', 'care', 'financial', 'problemi', 'help']",0
4,"['might', 'ax', 'well']",0
...,...,...
4006,"['dear', 'uve', 'invited', 'xchat', 'final', '...",1
4007,"['oh', 'love', 'soooo', 'good', 'hear', 'omg',...",0
4008,"['get', 'gandhipuram', 'walk', 'cross', 'cut',...",0
4009,"['leaving', 'house']",0


In [61]:
valid

,X_val,y_val
0,"['r', 'sure', 'understand', 'wine', 'good', 'i...",0
1,"['wot', 'student', 'discount', 'get', 'book']",0
2,"['boy', 'loved', 'gal', 'propsd', 'bt', 'mind'...",0
3,"['going', 'dinnermsg']",0
4,"['pas', 'dis', 'contact', 'n', 'see', 'wat', '...",0
...,...,...
441,"['anyways', 'go', 'gym', 'whatever', 'love', '...",0
442,"['hi', 'dear', 'saw', 'dear', 'happy', 'batter...",0
443,"['kyou', 'girl', 'waiting', 'reception', 'ah']",0
444,"['ever', 'one', 'foot']",0


In [63]:
valid

,X_val,y_val
0,"['r', 'sure', 'understand', 'wine', 'good', 'i...",0
1,"['wot', 'student', 'discount', 'get', 'book']",0
2,"['boy', 'loved', 'gal', 'propsd', 'bt', 'mind'...",0
3,"['going', 'dinnermsg']",0
4,"['pas', 'dis', 'contact', 'n', 'see', 'wat', '...",0
...,...,...
441,"['anyways', 'go', 'gym', 'whatever', 'love', '...",0
442,"['hi', 'dear', 'saw', 'dear', 'happy', 'batter...",0
443,"['kyou', 'girl', 'waiting', 'reception', 'ah']",0
444,"['ever', 'one', 'foot']",0


In [64]:
mlflow.sklearn.autolog()

n_estimators = 200
max_depth = 5
clf = LogisticRegression(random_state=101)
clf.fit(X_train, train.y_train)

y_pred = clf.predict(X_val_tfidf)
aucpr = eval_metrics(valid.y_val, y_pred)
acc = accuracy_score(valid.y_val, y_pred)
conf_1=confusion_matrix(valid.y_val, y_pred)

with mlflow.start_run(run_name="logistic Regression"):
  
    
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.log_dict(np.array(conf_1).tolist(), "confusion_matrix.json")
    mlflow.sklearn.log_model(clf, "model")

    print("\nRandom Classifier Model (no_of_estimator={:f}, max_depth={:f}):".format(n_estimators, max_depth))
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr} ")
    print(f"Confusion Matrix: {conf_1} \n \n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="Logistic Regression"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="Logistic Regression")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/04/04 21:07:21 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/04/04 21:07:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3362332e6f894e8880b3a4c6d10bf95f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Random Classifier Model (no_of_estimator=200.000000, max_depth=5.000000):
Accuracy: 0.8811659192825112
AUCPR: 0.5594170403587444 
Confusion Matrix: [[393   0]
 [ 53   0]] 
 



Successfully registered model 'Logistic Regression'.
2023/04/04 21:07:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Logistic Regression, version 1
Created version '1' of model 'Logistic Regression'.


In [65]:
mlflow.tracking.MlflowClient().get_model_version("Logistic Regression","1")

<ModelVersion: creation_timestamp=1680642449352, current_stage='None', description=None, last_updated_timestamp=1680642449352, name='Logistic Regression', run_id='c457d56a40ae4aefb8dfe5d368a0fcc1', run_link=None, source='file:///content/mlruns/0/c457d56a40ae4aefb8dfe5d368a0fcc1/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>

## Model 2 :multinomial Naive Bayes

In [ ]:
spam_detector_MNB = MultinomialNB().fit(X_train_tfidf, train.y_train)

In [83]:
mlflow.sklearn.autolog()

clf = MultinomialNB()
clf.fit(X_train_tfidf, train.y_train.values)
#spam_detector_MNB = MultinomialNB().fit(X_train_tfidf, train.y_train)
y_pred = clf.predict(X_val_tfidf)
aucpr = eval_metrics(valid.y_val, y_pred)
acc = accuracy_score(valid.y_val, y_pred)
conf_2=confusion_matrix(valid.y_val, y_pred)

with mlflow.start_run(run_name="Multinomial Naive Bayes"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.sklearn.log_model(clf, "model")
    mlflow.log_dict(np.array(conf_2).tolist(), "confusion_matrix.json")

    print("\nMultinomial Naive Bayes")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr}")
    print(f"Confusion Matrix: {conf_2} \n\n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="gaussian-nb-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="gaussian-nb-model")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/04/04 21:18:23 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/04/04 21:18:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8f466de21fa940c4953578959958dbdf', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multinomial Naive Bayes
Accuracy: 0.968609865470852
AUCPR: 0.8836195955664607
Confusion Matrix: [[393   0]
 [ 14  39]] 




Successfully registered model 'gaussian-nb-model'.
2023/04/04 21:18:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: gaussian-nb-model, version 1
Created version '1' of model 'gaussian-nb-model'.


## Model 03 : multilayer perceptron

In [85]:
mlflow.sklearn.autolog()

clf = MLPClassifier(random_state=101,learning_rate='adaptive')
clf.fit(X_train_tfidf, train.y_train.values)

y_pred = clf.predict(X_val_tfidf)
aucpr = eval_metrics(valid.y_val, y_pred)
acc = accuracy_score(valid.y_val, y_pred)
conf_3=confusion_matrix(valid.y_val, y_pred)

with mlflow.start_run(run_name="Multilayer Perceptron"):

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("AUCPR",aucpr)
    mlflow.sklearn.log_model(clf, "model")
    
    print("\nMultilayer Perceptron")
    print(f"Accuracy: {acc}")
    print(f"AUCPR: {aucpr}")
    print(f"Confusion Matrix {conf_3} \n\n")
    

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="sklearn-model",
        registered_model_name="multilayer-perceptron-model"
    )
    if tracking_url_type_store != "file":
      mlflow.sklearn.log_model(clf, "model", registered_model_name="multilayer-perceptron-model")
    else:
      mlflow.sklearn.log_model(clf, "model")

2023/04/04 21:21:16 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/04/04 21:21:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3e636eba7a7d4f99aa82a4b0764d7302', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow



Multilayer Perceptron
Accuracy: 0.9887892376681614
AUCPR: 0.9584355698451645
Confusion Matrix [[393   0]
 [  5  48]] 




Successfully registered model 'multilayer-perceptron-model'.
2023/04/04 21:21:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: multilayer-perceptron-model, version 1
Created version '1' of model 'multilayer-perceptron-model'.


In [86]:
mlflow.tracking.MlflowClient().get_model_version("multilayer-perceptron-model","1")

<ModelVersion: creation_timestamp=1680643309796, current_stage='None', description=None, last_updated_timestamp=1680643309796, name='multilayer-perceptron-model', run_id='86527b21424f4fdda6f873cee19d9bf6', run_link=None, source='file:///content/mlruns/0/86527b21424f4fdda6f873cee19d9bf6/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>